# TPSC and the Mermin-Wagner theorem

$$ \chi_{sp}(k, i\omega_n) \equiv \frac{\chi_0(k, i\omega_n)}{1 - \frac{U_{sp}}{2} \chi_0(k, i\omega_n)} $$

$$ \chi_{ch}(k, i\omega_n) \equiv \frac{\chi_0(k, i\omega_n)}{1 + \frac{U_{ch}}{2} \chi_0(k, i\omega_n)} $$

$$ \mathrm{Tr} [ \chi ] \equiv \frac{1}{\beta N_k} \sum_{n, k} \chi(k, i\omega_n) $$


Temperature sweep for $U=4$

Spin structure factor 

$$S(k) \equiv \sum_n \chi_{sp}(i\omega_n, k)$$

In [1]:
from pytriqs.plot.mpl_interface import plt
import numpy as np
from math import cos, pi
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, Gf, MeshProduct, Idx
from pytriqs.archive import HDFArchive

In [4]:
%reload_ext cpp2py.magic

In [5]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp>
using namespace triqs::gfs;

// The type of a Green function : (k,omega) -> Complex number
using g_k_w_type = gf_view<cartesian_product<brillouin_zone, imfreq>, scalar_valued>;
using g_r_t_type = gf<cartesian_product<cyclic_lattice, imtime>, scalar_valued>;

g_k_w_type bubble(g_k_w_type g0) {
    
    // Fourier Transformation of k, \omega to obtain g(r,t)
    auto grt = make_gf_from_fourier<0,1>(g0);
    
    // The mesh of gtr is a cartesian product mt x mr. We decompose it.
    auto [mr, mt] = grt.mesh();
    
    // The inverse temperature from the mesh
    double beta = mt.domain().beta;
    
    // A new mesh for chi, with a bosonic statistics, but same size as mt.
    auto mtb = gf_mesh<imtime>{beta, Boson, mt.size()};
    
    // Build chi (r, tau) with this new mesh.
    auto chi0 = g_r_t_type{{mr, mtb}};

    // we fill chi : chi(tau, r) = g(beta - tau, -r) * g(tau, r)
    for (auto const &r : mr)      
        for (auto const &t : mtb) 
            chi0[r, t] = grt(-r, beta - t) * grt(r, t); 

    // Fourier transform back to k, \omega space and return
    return make_gf_from_fourier<0,1>(chi0);
}

In [60]:
def eps(k):
    return -2 * t* (cos(k[0]) + cos(k[1]))

def eps2(kx,ky):
    return -2 * t* (cos(kx) + cos(ky))

t = 1.0
n_k = 128
n_w = 128
iGamma = 0.01 * 1j

# Two unit vectors in R3
BL = BravaisLattice([(1, 0, 0), (0, 1, 0)])
BZ = BrillouinZone(BL)

# A regular mesh on the BZ with 30 x 30 points
kmesh = MeshBrillouinZone(BZ, n_k = n_k)
wmesh = MeshImFreq(beta=beta, S='Fermion', n_max=n_w)

# Init g0
g0 = Gf(mesh = MeshProduct(kmesh, wmesh),target_shape = [])
    
def fill_g0(g0, beta):
    w = np.tensordot(np.ones(n_k*n_k), list(wmesh.values()), 0)
    k = np.tensordot(list(kmesh.values()), np.ones(2*n_w), 0)
    g0.data[:,:] = 1 / (w - np.vectorize(eps2)(k[:,0,:], k[:,1,:]) + iGamma)


def fill_g0_2(g0, beta):
    for k in g0.mesh[0]:
        for w in g0.mesh[1]:
            g0[k,w] = 1/(w - eps(k) + iGamma)


def get_chi0(g0):
    return bubble(g0)

def chi_rpa(chi0_wk, U):
    chi_rpa = chi0_kw.copy()
    chi_rpa.data[:,:] = chi0_kw.data[:,:] / (1 - U * chi0_kw.data[:,:])
    return chi_rpa

In [63]:
%%timeit
fill_g0(g0, 1)

1 loop, best of 3: 3.19 s per loop


In [ ]:
%%timeit
fill_g0_2(g0, 1)

In [40]:
a = np.zeros([10,100,5])
b = np.roll(a,0)
np.swapaxes(a, 1, 2).shape
a.shape

(10, 100, 5)

In [10]:
t = 1.0
n_k = 128
n_w = 128
iGamma = 0.01 * 1j
U = 4.

T_rpa_vec = np.concatenate((np.arange(10., 3., -1.), np.arange(3., 0.75, -0.2)))
S_rpa_vec = np.zeros_like(T_rpa_vec)

print ''.join('| %-11s' % s for s in ['T', 'beta', 'S_rpa']), '|'
print '-'*41

for idx, T in enumerate(T_rpa_vec):

    beta = 1. / T
    g0_kw = get_g0(beta, t, n_k, n_w, iGamma)
    chi0_kw = get_chi0(g0_kw)

    chi = chi_rpa(chi0_kw, 0.5*U)
    
    S_rpa = chi[Idx(64,64,0),:].data.sum().real # FIXME: Replace with .density() call
    S_rpa_vec[idx] = S_rpa
    
    print ''.join('| %4.4E ' % x for x in [T, beta, S_rpa]), '|'

| T          | beta       | S_rpa       |
-----------------------------------------
| 1.0000E+01 | 1.0000E-01 | 2.6552E-02  |
| 9.0000E+00 | 1.1111E-01 | 2.9733E-02  |
| 8.0000E+00 | 1.2500E-01 | 3.3785E-02  |
| 7.0000E+00 | 1.4286E-01 | 3.9127E-02  |
| 6.0000E+00 | 1.6667E-01 | 4.6488E-02  |
| 5.0000E+00 | 2.0000E-01 | 5.7283E-02  |
| 4.0000E+00 | 2.5000E-01 | 7.4614E-02  |
| 3.0000E+00 | 3.3333E-01 | 1.0673E-01  |
| 2.8000E+00 | 3.5714E-01 | 1.1666E-01  |
| 2.6000E+00 | 3.8462E-01 | 1.2855E-01  |
| 2.4000E+00 | 4.1667E-01 | 1.4299E-01  |
| 2.2000E+00 | 4.5455E-01 | 1.6084E-01  |
| 2.0000E+00 | 5.0000E-01 | 1.8336E-01  |


KeyboardInterrupt: 

In [9]:
chi_rpa(chi0_kw, 0.5*U)[]

Green Function  with mesh Brillouin Zone Mesh  with linear dimensions (128 128 1), Domain: Brillouin Zone with repiprocal matrix 
[[6.28319,0,0]
 [0,6.28319,0]
 [0,0,6.28319]], Matsubara Freq Mesh of size 255, Domain: Matsubara domain with beta = 0.1, statistic = Boson, positive_only : 0 and target_rank 0: 

In [ ]:
n = 1.
U = 4.

Usp, Uch = 1., 1. # initial guess

T_tpsc_vec = np.array([10., 9., 8., 7., 6., 5., 4., 3., 
                       2.5, 2.0, 1.5, 1.2, 1.0, 
                       0.8, 0.6, 0.4, 0.35, 0.3, 0.25])

S_tpsc_vec = np.zeros_like(T_tpsc_vec)
U_sp_vec = np.zeros_like(T_tpsc_vec)

print ''.join('| %-11s' % s for s in ['T', 'beta', 'Usp', 'Uch', 'docc', 'S_tpsc']), '|'
print '-'*80

for idx, T in enumerate(T_tpsc_vec):

    beta = 1. / T    
    g0_wk, g0_wr, g0_tr = get_g0(e_k, beta, mu, n_w, kmesh, rmesh)
    chi0_tr, chi0_wr, chi0_wk = get_chi0(g0_tr, n_w, kmesh, rmesh)    
    
    Usp, Uch, docc = solve_Usp_and_Uch(chi0_wk, U, n, Usp0=Usp, Uch0=Uch)
    
    chi_sp_wk = chi_wk_from_U_and_chi0_wk(chi0_wk, 0.5*Usp)
    S_tpsc = chi_sp_wk[:, k_pipi].data.sum().real

    S_tpsc_vec[idx], U_sp_vec[idx] = S_tpsc, Usp

    print ''.join('| %4.4E ' % x for x in [T, beta, Usp, Uch, docc, S_tpsc]), '|'


In [ ]:
plt.figure(figsize=(3.25*2, 5*4))
subp = [4, 1, 1]

plt.subplot(*subp); subp[-1] += 1
plt.title(r'$U = %2.2f$' % U)
plt.plot(T_rpa_vec, S_rpa_vec, 'o-', label=r'$S_{RPA}$', alpha=0.5)
plt.plot(T_tpsc_vec, S_tpsc_vec, 'o-', label=r'$S_{TPSC}$', alpha=0.5)
plt.legend(loc='best')
plt.xlabel(r'$T$')

plt.subplot(*subp); subp[-1] += 1
plt.plot(T_rpa_vec, 1./S_rpa_vec, 'o-', alpha=0.5, label=r'$S_{RPA}^{-1}$')
plt.plot(T_tpsc_vec, 1./S_tpsc_vec, 'o-', alpha=0.5, label=r'$S_{TPSC}^{-1}$')
plt.legend(loc='best')
plt.xlabel(r'$T$'); plt.grid()

plt.subplot(*subp); subp[-1] += 1
plt.plot(T_rpa_vec, 1./S_rpa_vec, 'o-', alpha=0.5, label=r'$S_{RPA}^{-1}$')
plt.plot(T_tpsc_vec, 1./S_tpsc_vec, 'o-', alpha=0.5, label=r'$S_{TPSC}^{-1}$')
plt.legend(loc='best'); plt.xlim([0, 2]); plt.ylim([-0.1, 2.5])
plt.xlabel(r'$T$'); plt.grid()

plt.subplot(*subp); subp[-1] += 1
plt.plot(T_rpa_vec, U + 0*T_rpa_vec, 'o-', alpha=0.5, label=r'$U$')
plt.plot(T_tpsc_vec, U_sp_vec, 'o-', alpha=0.5, label=r'$U_{sp}$')
plt.legend(loc='best')
plt.xlabel(r'$T$')

plt.tight_layout()
plt.savefig('figure_S_TPSC_RPA_vs_T.pdf')

# RPA phase boundary

While the TPSC fulfills the Mermin-Wagner theorem RPA does not. Instead it predicts an anti-ferro magnetic transition at finite temperature for finite interaction $U$.


In [ ]:
Tc_rpa = np.arange(0.01, 1.0, 0.05)
Uc_rpa = np.zeros_like(Tc_rpa)

for idx, T in enumerate(Tc_rpa):

    beta = 1. / T    
    g0_wk, g0_wr, g0_tr = get_g0(e_k, beta, mu, n_w, kmesh, rmesh)
    chi0_tr, chi0_wr, chi0_wk = get_chi0(g0_tr, n_w, kmesh, rmesh)    
        
    chi0_w0kpipi = chi0_wk[Idx(0), k_pipi][0, 0].real
    Uc = 2.0 / chi0_w0kpipi
    Uc_rpa[idx] = Uc
    
    print 'T, beta, Uc =', T, beta, Uc
        

In [ ]:
plt.plot(Uc_rpa, Tc_rpa, '.-')